In [1]:
import pandas as pd

# Read the Excel workbook
df = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls')

# Load Access, Stores, Assistance, Insecurity, Local, Health, Restaurants, Socioeconomic data sheets as dataframes 
access = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ACCESS')
stores = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'STORES')
assistance = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'ASSISTANCE')
insecurity = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'INSECURITY')
prices_taxes = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'PRICES_TAXES')
local = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'LOCAL')
health = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'HEALTH')
restaurants = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'RESTAURANTS')
socioeconomic = pd.read_excel('data/2018-usda-food-environment-atlas-dataset.xls', 'SOCIOECONOMIC')

In [3]:
# We're going to join all these sheets, so we'll drop redundant state and county cols from all but one.
# We'll also set the FIPS ID col to be index so we can use that for the join.
dfs = [stores, assistance, insecurity, prices_taxes, local, health, restaurants, socioeconomic]
for df in dfs:
    df.drop(columns=['State', 'County'], axis=1, inplace=True)
    df.set_index('FIPS', inplace=True)

# Then, we'll also set the index on the Access df. This will be the dataframe we join the others onto.
access.set_index('FIPS', inplace=True)

# Combine all sheets into one dataframe by joining on FIPS col.
# Now we have a master dataframe containing the cols from all sheets.
master_df = access.join(dfs)

# These are the features I was interested in. Feel free to change them aroud as you see fit.
# The sheeet 'Variable List' contains descriptions of all available vars.
cols_of_interest = ['State', 'County',
                    'PCT_LACCESS_POP15','PCT_LACCESS_LOWI15','PCT_LACCESS_HHNV15','PCT_LACCESS_SNAP15',
                    'PCT_LACCESS_CHILD15','PCT_LACCESS_SENIORS15','PCT_LACCESS_WHITE15','PCT_LACCESS_BLACK15',
                    'PCT_LACCESS_HISP15','PCT_LACCESS_NHASIAN15','PCT_LACCESS_NHNA15','PCT_LACCESS_NHPI15',
                    'PCT_LACCESS_MULTIR15','GROCPTH14','SUPERCPTH14','CONVSPTH14','SPECSPTH14','SNAPSPTH16',
                    'FFRPTH14','FSRPTH14','PCT_SNAP16','PCT_NSLP15','PCT_SBP15','PCH_SFSP_09_15','PCT_SFSP15',
                    'PCT_WIC15','SODATAX_STORES14','SODATAX_VENDM14','CHIPSTAX_STORES14','CHIPSTAX_VENDM14',
                    'FOOD_TAX14','FMRKTPTH16','PCT_FMRKT_SNAP16','PCT_FMRKT_WIC16','PCT_FMRKT_WICCASH16',
                    'PCT_FMRKT_SFMNP16','PCT_FMRKT_CREDIT16','PCT_FMRKT_FRVEG16','PCT_FMRKT_ANMLPROD16',
                    'PCT_FMRKT_BAKED16','PCT_FMRKT_OTHERFOOD16','FOODHUB16','PCT_DIABETES_ADULTS13',
                    'PCT_OBESE_ADULTS13','PCT_HSPA15','RECFACPTH14','PCT_NHWHITE10','PCT_NHBLACK10',
                    'PCT_HISP10','PCT_NHASIAN10','PCT_NHNA10','PCT_NHPI10','PCT_65OLDER10','PCT_18YOUNGER10',
                    'MEDHHINC15','POVRATE15','METRO13']

# Create a dataframe with just the cols of interest.
combined_df = master_df[cols_of_interest]

# Export the combined df for visualization
combined_df.to_csv('data/combined.csv', index=False)
combined_df.dtypes

In [6]:
import seaborn as sns
from sklearn import linear_model
from sklearn.metrics import r2_score
from sklearn.model_selection import train_test_split

# features_new = stores_cols + assistance_cols + local_cols + health_cols + restaurant_cols + socioeconomic_cols + prices_taxes_cols + access_cols
# prediction_features = list(set(features_new) - set(['FIPS', 'State', 'County', 'PCT_DIABETES_ADULTS13', 'PCT_OBESE_ADULTS13']))
prediction_features = ['FFRPTH14','FMRKTPTH16','FOOD_TAX14']
target = 'PCT_OBESE_ADULTS13'

dfs = [stores, assistance, insecurity, local, health, restaurants, socioeconomic, prices_taxes]
for df in dfs:
    df.set_index('FIPS', inplace=True)
access.set_index('FIPS', inplace=True)
df = access.join(dfs)

# df_cols = list(set(features_new) - set(['FIPS']))
# df = master_df[df_cols]

# df = master_df[prediction_features]
df = df.dropna()
df.isnull().sum()

# df['METRO13'] = df['METRO13'].astype('bool')

# Split the data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(df[prediction_features], df[target], test_size=0.33)
regr = linear_model.LinearRegression()
regr.fit(X_train, y_train)
y_pred = regr.predict(X_test)

print(r2_score(y_train, regr.predict(X_train)))
print(r2_score(y_test, y_pred))

# Exporting the model for prediction calculator
# from sklearn.externals import joblib
# joblib.dump(regr, 'regr.pkl')

KeyError: 'FIPS'